# Local View Maker

## [Step 1 - Setting Global Variable](#1---Setting-Global-Variable)

## [Step 2 - Load Map](#2---Load-Map)

## [Step 3 - Start Simulation](#3---Start-Simulation)

## Requirements

- ipywidgets==7.6.2
- plotly
- pandas
- numpy
- sympy

In [1]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import time
import pandas as pd
from random import randrange
import random
import math
from sympy import *
import csv

## 1 - Setting Global Variable

In [2]:
# Global Variable

# distance unit: nm
# speed unit:    knot

# time_interval: 6 min
time_interval = 6

# each block is 6 nm
map_block_distance = 6
view_size = 18

view_map = []
view_point = (18,18)
voyage_plan = [(1,5),(10,14),(5,30)]

# own ship
guard_ring_size = 12

# other ship
buffer_size = 6

# record
is_record = 0

## Data Structure

### Ship
- x, y, course, speed

#### OwnShip
- attribute: rpm, rudder angle, draught, waypoint_index
- function: waypoint

#### OtherShip
- function: cpa, tcpa


In [3]:
# Parent Class

class Ship:
    def __init__(self, x, y, course, speed):
        self.x = int(x) 
        self.y = int(y)
        self.course = course
        self.speed = speed
    
    def print_info(self):
        print(f"position:\t({self.x},{self.y})\r\ncourse:\t\t{self.course} °\r\nspeed:\t\t{self.speed} kn")

    def position(self):
        return (self.x,self.y)

    def update_course(self, course):
        self.course = course    
    def update_speed(self, speed):
        self.speed = speed
        
# Child Class

class OwnShip(Ship):
    rpm = 0
    rudder_angle = 0
    
    draught = 0
    waypoint_index = 1
    
    def waypoint(self):
        return voyage_plan[self.waypoint_index]

    def update_course(self, course):
        self.course = course 
    
class OtherShip(Ship):
    tcpa = 0
    cpa = 0
    
    # minute    
    def update_tcpa(self, target_ship):
        # (x, y, course, speed)
        a = (target_ship.x, target_ship.y, target_ship.course, target_ship.speed)
        b = (self.x, self.y, self.course, self.speed)
        
        s1 = (round(a[3]*math.sin(a[2]*math.pi/180),0),round(a[3]*math.cos(a[2]*math.pi/180),0))
        s2 = (round(b[3]*math.sin(b[2]*math.pi/180),0),round(b[3]*math.cos(b[2]*math.pi/180),0))

        t = symbols('t')

        aa = (a[0] + s1[0]*t, a[1] + s1[1]*t )
        bb = (b[0] + s2[0]*t, b[1] + s2[1]*t )
        dis = (aa[0]-bb[0])*(aa[0]-bb[0]) + (aa[1]-bb[1])*(aa[1]-bb[1])
        dis = diff(dis)
        dis
        t = solve(dis, t)
        tcpa = []
        for i in t:
            if i > 0:
                tcpa.append(round(i,0))
    
        if not tcpa:
            tcpa = -1
        else:
            tcpa = round(min(tcpa)*60,2)

        self.tcpa = tcpa
        
    # nm
    def update_cpa(self, target_ship):
        a = (target_ship.x, target_ship.y, target_ship.course, target_ship.speed)
        b = (self.x, self.y, self.course, self.speed)
        s1 = (round(a[3]*math.sin(a[2]*math.pi/180),0),round(a[3]*math.cos(a[2]*math.pi/180),0))
        if self.tcpa == -1:
            aa = (b[0],b[1])
        else:
            aa = (a[0] + s1[0]*self.tcpa, a[1] + s1[1]*self.tcpa )
    #     bb = (b[0] + s2[0]*tcpa, b[1] + s2[1]*tcpa )
        cpa = round(math.sqrt(math.pow(a[0]-aa[0],2) + math.pow(a[1]-aa[1],2)),2)
        self.cpa = cpa*map_block_distance
    

In [4]:
# example 

a = OwnShip(1,2,3,4)

a.print_info()
a.waypoint_index = 1
a.waypoint()
b = a.waypoint()
b[0]

position:	(1,2)
course:		3 °
speed:		4 kn


10

### Record Data

| step | Own Ship | x | y | course | speed | rpm | rudder angle | waypoint | cpa | tcpa |
| :--: | :------: |:-:|:-:| :----: | :---: | :-: | :----------: | :------: | :-: | :--: |
|0|Own ship|1|1|90|6|3000|10|(10, 14)|||
|1|Other ship|2|3|270|8||||3|10|


### Color

|Color|value|
| :-: | :-: |
|Green|8|
|Yellow|5|
|Red|3|
|White|0|
|Blue|-2|
|Brown|-5|
|Grey|-7|
|Black|-10|

In [5]:
# Color

Color_Green = 8
Color_Yellow = 5
Color_Red = 3
Color_White = 0
Color_Blue = -2
Color_Brown = -5
Color_Grey = -7
Color_Black = -10


## Function Define
### Data Function

In [6]:
# data function

# save data
# make sure document and file are existed
def save_NP_data(np_vals, file_name):
    pd_vals = pd.DataFrame(np_vals)

    
    pd_vals.to_csv(file_name, header=0, index=0)
    
    
    
# load data
def load_NP_data(file_name):
    load_pd_vals = pd.read_csv(file_name,header=None)
    return load_pd_vals



# resize data
def resize_NP_data(np_vals, resize):
    resize_vals = np.zeros((resize[0],resize[1]),dtype=int)
    
    for j in range(np_vals.shape[1]):
        for i in range(np_vals.shape[0]):
            resize_vals[j][i] = np_vals[i][j]
    return resize_vals

### Map Function

In [7]:
# Map Function

# init map
# load map

# init map
def init_map(x_size, y_size):
    map = np.zeros((y_size, x_size),dtype=int)
    return map

# load map
def load_map(file_name):
    return load_NP_data(file_name).to_numpy()




In [8]:
# map function test

# aa = init_map(3,2)
# aa

### Render Function

In [9]:
# render function - 1

def handle_click(trace, points, state):
    print(points.point_inds)

def view_map_show(map):
#     px.on_click(handle_click)

    # 黑、灰、棕、藍、白、紅、黃、綠
    fig = px.imshow(map,
                    range_color=[-10, 10],
                    color_continuous_scale=[(0.00, "black"),   (0.125, "black"),
                                            (0.125, "grey"),   (0.25, "grey"),
                                            (0.25, "#E0AE44"),   (0.375, "#E0AE44"),
                                            (0.375, "#E0FFFF"),   (0.5, "#E0FFFF"),
                                            (0.5, "white"),   (0.625, "white"),
                                            (0.625, "red"), (0.75, "red"),
                                            (0.75, "#FFCC00"), (0.875, "#FFCC00"),
                                            (0.875, "green"),  (1.00, "green")])
    
    fig = go.FigureWidget(fig)
    return fig
    

# render - ship

def buffer(map, point, size):
    pass


In [10]:
# render function - 2

# view point change
def view_point_change(map_vals, view_point):
    view_x, view_y = list(view_point)
    return map_vals[view_x-view_size:view_x+view_size+1, view_y-view_size:view_y+view_size+1]

def render(map_vals, render_datas, color):
    list_data = list(render_datas)
    
    for data in list_data:
        x,y = list(data)
        x = int(round(x,0))
        y = int(round(y,0))
        shape_x, shape_y = map_vals.shape
        if x< shape_x and y < shape_y and x >=0 and y >= 0:
            map_vals[y][x] = color
        
    return map_vals

def neighbor_points(point):
    neighbors = []
    startx, starty = list((point[0]-1,point[1]-1))

    for i in range(3):
        neighbors.append((startx, starty+i))
    neighbors.append((startx+1, starty))
    neighbors.append((startx+1, starty+2))
    for i in range(3):
        neighbors.append((startx+2, starty+i))
    
    return neighbors

def distance_a_b(a,b):
    return round(math.sqrt(math.pow(a[0]-b[0],2) + math.pow(a[1]-b[1],2)),2)

def is_neighbor(pa, pb):
    if(pa == pb):
        return True
    
#     neighbors = neighbor_points(pb)
#     for neighbor in neighbors:
#         if(pa == neighbor):
#             return True

    distance = distance_a_b(pa,pb)
    # about 1 block length (6 nm)     
    if(distance < 1):
        return true
    return False

def render_view_map(view_map_vals):
    datas = [(1,1),(2,2)]
    # render map - Color Brown
#     view_map_vals = render(view_map_vals, datas, Color_Brown)
    
    # render depth of water - Color Blue
#     view_map_vals = render(view_map_vals, datas, Color_Blue)
    
    # render voyage plan
#     view_map_vals = render(view_map_vals, voyage_plan, Color_Green)
    for i in voyage_plan:
        neighbors = neighbor_points(i)
        view_map_vals = render(view_map_vals, neighbors, Color_Grey)
    
    # render ship - Color Green/Black
    view_map_vals = render(view_map_vals, [own_ship.position()], Color_Green)
    for i in other_ships:
#         print(i.position())
        view_map_vals = render(view_map_vals, [i.position()], Color_Black)
    
    # render ship buffer - Color Red/Grey
#     view_map_vals = render(view_map_vals, datas, Color_Red)
#     view_map_vals = render(view_map_vals, datas, Color_Grey)
    
    # render heading line
    
    return view_map_vals
#     pass

### Control Function

In [11]:
# control function

# update position
def update_position(x, y, course, speed):
    xx = round(speed*math.sin(course*math.pi/180)*time_interval/60/map_block_distance+x,3)
    yy = round(speed*math.cos(course*math.pi/180)*time_interval/60/map_block_distance+y,3)
    return xx, yy

# rpm with speed (time)
# rudder angle with course (time & rpm)

# other ship
# fixed course、speed、rpm、rudder angle

def included_angle(x1,y1,x2,y2):
    x = x2-x1
    y = y2-y1

    ans= (math.atan(y/x))*180/math.pi
    ans = round(ans,0)
    if(x1>x2):
        ans = 270-ans
    else:
        ans = 90-ans
    return ans

def fixed_course(x1,y1,x2,y2):
    course = included_angle(x1,y1,x2,y2)
    return course

def fixed_speed(speed):
    return speed

def fixed_rpm(rpm):
    return rpm

def fixed_rudder_angle(rudder_angle):
    return rudder_angle

# FIXME

def dynamic_course(course):
    pass

def dynamic_speed(speed):
    pass

# own ship
# general / special navigation

# course: degree

def general_navigation():
    pass
    
# FIXME
def special_navigation():
    pass


## 2 - Load Map

In [12]:
map_file = './data/37x37-test_map.csv'
map_vals = load_map(map_file)

In [13]:
print(type(map_vals))
map_vals.shape

<class 'numpy.ndarray'>


(37, 37)

## 3 - Start Simulation

### Variable

- map_vals
- view_map_vals
- temp_map_vals
--- 
- view_map
- view_map_data

In [14]:
# random other ship
random.seed(2)

random_count = 5
test_other_ships = []

for i in range(random_count):
    x = randrange(view_size*2)
    y = randrange(view_size*2)
    course = randrange(3600)/10
    speed = randrange(60)/10
    test_other_ships.append(OtherShip(x,y,course,speed))
#     print(i)
#     test_other_ships[i].print_info()
#     print()

In [15]:
# Start Simulation
change_view_point = 0

own_ship = OwnShip(1,5,0,6)

# other ship
other_ships = test_other_ships


In [16]:
# test

# for i in other_ships:
#     print(i.x,i.y)
#     print(i.position())
#     i.print_info()



# for ship in other_ships:
#     ship.x, ship.y = update_position(ship.x, ship.y, ship.course, ship.speed)
#     print(ship.position())

    
    
# own_ship.position()

![chart](data/sim_ship.png)

In [21]:
# Start Simulation

# while arrive destination

# view_map_vals = view_point_change(map_vals, view_point)
step = 0
record_datas = []
while(1):
# for loop in range(10):

    # if change view point
    if(change_view_point):
        view_map_vals = view_point_change(map_vals, view_point)

    # update position
    own_ship.x, own_ship.y = update_position(own_ship.x, own_ship.y, own_ship.course, own_ship.speed)

    for ship in other_ships:
        ship.x, ship.y = update_position(ship.x, ship.y, ship.course, ship.speed)

    # render view map
    temp_map_vals = view_map_vals.copy()
    temp_map_vals = render_view_map(temp_map_vals)

    # update waypoint_index
    if(is_neighbor(own_ship.position(),own_ship.waypoint())):
        if(own_ship.waypoint_index == (len(voyage_plan)-1)):
            print("Finish !!!")
            break
        own_ship.waypoint_index = own_ship.waypoint_index+1
#         print(own_ship.waypoint_index)
#         print(own_ship.waypoint())
    
    # update other ship cpa、tcpa
    for ship in other_ships:
        ship.update_tcpa(own_ship)
        ship.update_cpa(own_ship)
#         print(ship.tcpa, '\t', ship.cpa)

    # update course、speed、rpm、rudder angle(rpm, rudder angle)
    for i in other_ships:
        temp_course = i.course
        temp_speed = i.speed
        i.update_course(temp_course)
        i.update_speed(temp_speed)
    
    # update own ship course、speed
    temp_waypoint = own_ship.waypoint()
    temp_course = fixed_course(own_ship.x, own_ship.y, temp_waypoint[0], temp_waypoint[1])
    temp_speed = own_ship.speed
    own_ship.update_course(temp_course)
    own_ship.update_speed(temp_speed)

    # record data - 1 
    own_ship_data = [step,'Own Ship', own_ship.x, own_ship.y, own_ship.course, own_ship.speed, own_ship.rpm, own_ship.rudder_angle, own_ship.waypoint(),'','']
#     print(own_ship_data)
    record_datas.append(own_ship_data)
    other_ships_index = 0
    for i in other_ships:
        other_ship_data = [step,'Other Ship - '+ str(other_ships_index), i.x, i.y, i.course, i.speed, '', '', '', i.cpa, i.tcpa]
#         print(other_ship_data)
        record_datas.append(other_ship_data)
        other_ships_index += 1
    # draw map
    view_map_data.z = temp_map_vals
    
    step += 1
    time.sleep(0.001)

# record data - 2
if(is_record):
    t = time.localtime()
    current_time = time.strftime("%m-%d-%H:%M", t)
    file_name = 'record/record - ' + current_time + 'UTC.csv'
    with open(file_name, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(record_datas)

Finish !!!


### Animate

In [18]:
# First Part
view_map_vals = view_point_change(map_vals, view_point)
temp_map_vals = view_map_vals.copy()

view_map = view_map_show(temp_map_vals)
view_map_data = view_map.data[0]

In [22]:
view_map_data.z = temp_map_vals
view_map

FigureWidget({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<br>y: %{y}<br>c…

In [23]:
# debug
print(step)
# own_ship.print_info()
# print(own_ship.waypoint())
# print(own_ship.waypoint_index)

279
